In [ ]:
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.common import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
driver = uc.Chrome()
wait = WebDriverWait(driver, timeout=10, poll_frequency=0.2)

In [ ]:
url = "https://map.naver.com/p"
driver.get(url)

In [ ]:
# search query

def search_query(query):
    query.replace(' ', '%20')
    url = f"https://map.naver.com/p/search/{query}"
    driver.get(url)

search_query("강남 음식점")

In [ ]:
# get objects
search_iframe = driver.find_element(By.ID, "searchIframe")
driver.switch_to.frame(search_iframe)

restaurant_list_scroll_container = driver.find_element(By.ID, '_pcmap_list_scroll_container')

In [ ]:
# dynamic scroll

scroll_num = 0
prev_scroll_remain = None
while True:
    scroll_top = driver.execute_script("return arguments[0].scrollTop", restaurant_list_scroll_container)
    scroll_height = driver.execute_script("return arguments[0].scrollHeight", restaurant_list_scroll_container)
    scroll_remain = scroll_height - scroll_top
    print(f"scroll_num: {scroll_num}, scroll_height: {scroll_height}, scroll_remain: {scroll_remain}")

    # end-of-loop
    if scroll_remain <= 0 or scroll_num >= 20 or prev_scroll_remain == scroll_remain:
        break

    # do scroll
    for _ in range(10):
        driver.execute_script(f"arguments[0].scrollTop += {scroll_remain // 10}", restaurant_list_scroll_container)
        time.sleep(0.1)

    # delay
    time.sleep(0.4)

    scroll_num += 1
    prev_scroll_remain = int(scroll_remain)

In [82]:
driver.switch_to.default_content()
driver.switch_to.frame(search_iframe)

In [84]:
restaurant_li_list = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li")
restaurant_a_list = driver.find_elements(By.CSS_SELECTOR, "div.CHC5F > div.bSoi3 > a")
prev_restaurant_iframe_src = None

for a in restaurant_a_list[:3]:
    a.click()

    # delay
    driver.switch_to.default_content()
    restaurant_iframe_locator = (By.XPATH, '//*[@id="entryIframe"]')
    try:
        wait.until(
            EC.frame_to_be_available_and_switch_to_it(restaurant_iframe_locator)
        )
    except:
        time.sleep(2)
        # driver.switch_to.default_content()

        driver.switch_to.frame(driver.find_element(*restaurant_iframe_locator))
        print('switched')

    # get info
    # restaurant_info : restaurant_name, restaurant_category, restaurant_address, restaurant_tel
    # review_data : total, key...

    # get restaurant_info
    for i in [
        ["restraunt_name", (By.CSS_SELECTOR, "#_title > div > span.GHAhO")],
        ["restraunt_category", (By.CSS_SELECTOR, "#_title > div > span.lnJFt")],
        ["restraunt_address", (By.CSS_SELECTOR, "div > a > span.LDgIH")],
        ["restraunt_tel", (By.CSS_SELECTOR, "div > span.xlx7Q")],
    ]:
        try:
            print(i[0], driver.find_element(*i[1]).text)
        except:
            print(i[0], None)

    # get review_data
    review_button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div[5]/div/div/div/div/a[5]')
    review_button.click()
    time.sleep(1)

    review_total = int(driver.find_element(By.XPATH, '//div[@class="jypaX"]/em').text[:-1].replace(",",""))

    # press review_showmore_button
    while True:
        review_showmore_button = driver.find_elements(By.XPATH, '//div[@class="IUbn3"]/a')

        if len(review_showmore_button) == 0:
            break

        review_showmore_button[0].click()
        time.sleep(0.5)


    review_li_list = driver.find_elements(By.XPATH, '//ul[@class="K4J9r"]/li')

    for review_li in review_li_list:
        l = review_li.text.strip().split("\n")
        print(l[0].replace("\"", ""), l[2])


    print('-----------')


    "#_title > div > span.GHAhO"
    "#_title > div > span.lnJFt"
    "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH"
    "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q"

    '//*[@id="app-root"]/div/div/div[5]/div/div/div/div/a[5]' # review_button

    '//*[@id="app-root"]/div/div/div[7]/div[3]/div[1]/div/div/div[1]/em' # review_total

    '//*[@id="app-root"]/div/div/div[7]/div[3]/div[1]/div/div/div[2]/div/a' # review_detail_button


    driver.switch_to.default_content()
    driver.switch_to.frame(search_iframe)


restraunt_name 디핀고
restraunt_category 와인
restraunt_address 서울 강남구 언주로150길 51 3층
restraunt_tel 0507-1371-8849
음식이 맛있어요 178
인테리어가 멋져요 127
특별한 메뉴가 있어요 124
특별한 날 가기 좋아요 112
친절해요 108
재료가 신선해요 55
매장이 청결해요 48
가성비가 좋아요 16
대화하기 좋아요 16
단체모임 하기 좋아요 14
기본 안주가 좋아요 10
주차하기 편해요 9
매장이 넓어요 9
술이 다양해요 8
차분한 분위기예요 7
음악이 좋아요 6
비싼 만큼 가치있어요 6
아늑해요 5
양이 많아요 5
고기 질이 좋아요 5
오래 머무르기 좋아요 5
컨셉이 독특해요 4
화장실이 깨끗해요 4
뷰가 좋아요 4
룸이 잘 되어있어요 3
혼밥하기 좋아요 2
메뉴 구성이 알차요 2
사진이 잘 나와요 2
건강한 맛이에요 2
좌석이 편해요 1
혼술하기 좋아요 1
음식이 빨리 나와요 1
현지 맛에 가까워요 1
반려동물과 가기 좋아요 1
-----------
restraunt_name 아르데레
restraunt_category 양식
restraunt_address 서울 강남구 압구정로42길 49 2층
restraunt_tel 0507-1425-6999
음식이 맛있어요 172
친절해요 98
인테리어가 멋져요 90
재료가 신선해요 73
특별한 메뉴가 있어요 73
매장이 청결해요 31
특별한 날 가기 좋아요 25
차분한 분위기예요 21
아늑해요 21
대화하기 좋아요 21
비싼 만큼 가치있어요 19
단체모임 하기 좋아요 19
화장실이 깨끗해요 14
메뉴 구성이 알차요 12
매장이 넓어요 12
디저트가 맛있어요 11
가성비가 좋아요 10
뷰가 좋아요 7
주차하기 편해요 5
건강한 맛이에요 4
양이 많아요 3
고기 질이 좋아요 3
사진이 잘 나와요 3
룸이 잘 되어있어요 3
술이 다양해요 2
커피가 맛있어요 2
음악이 좋아요 2
컨셉이 독특해요 1
음료가 맛있어요 1
음식이 빨리 나와요 1
--

In [ ]:
driver.switch_to.default_content()
driver.switch_to.frame(driver.find_element(*restaurant_iframe_locator))

# review_total = int(driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div[7]/div[3]/div[1]/div/div/div[1]/em').text)
review_total = int(driver.find_element(By.XPATH, '//div[@class="jypaX"]/em').text[:-1].replace(",",""))
print(review_total)

# review_showmore_button_locator = (By.XPATH, '//*[@id="app-root"]/div/div/div[7]/div[3]/div[1]/div/div/div[2]/div/a')

while True:
    review_showmore_button = driver.find_elements(By.XPATH, '//div[@class="IUbn3"]/a')

    if len(review_showmore_button) == 0:
        break

    review_showmore_button[0].click()

review_li_list = driver.find_elements(By.XPATH, '//ul[@class="K4J9r"]/li')
for review_li in review_li_list:
    l = review_li.text.strip().split("\n")
    print(l[0])
    l[2]

In [85]:
from tqdm import tqdm

for i in tqdm(range(1, 11)):
    for j in tqdm(range(100)):
        time.sleep(0.02)

 20%|██        | 2/10 [00:06<00:24,  3.12s/it]


KeyboardInterrupt: 